<a href="https://colab.research.google.com/github/bozorgpanah/WASP-Software-Engineering-Project/blob/main/Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow --upgrade

# Use seaborn for pairplot
!pip install seaborn

# Use some functions from tensorflow_docs
!pip install git+https://github.com/tensorflow/docs

In [ ]:
#Regression>>>predict the output of a continuous value
#Dataset>>>classic Auto MPG 
from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

print(tf.__version__)
name_db = "auto-mpg.data"
address_db = "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data"
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight', 'Acceleration', 'Model Year', 'Origin']
EPOCHS = 1000

#M
#Downloading the dataset
def downloading_dataset(name,address):
  dataset_path = keras.utils.get_file(name, address)
  return dataset_path

#A
#Importing the data using Pandas library
def preprocessing_dataset(dataset_path, columnNames):
 raw_dataset = pd.read_csv(dataset_path, names=columnNames, na_values = "?", comment='\t', sep=" ", skipinitialspace=True)
 dataset = raw_dataset.copy() 
 dataset.tail()
 #Cleanin the data
 dataset.isna().sum()
 dataset = dataset.dropna()
 dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})
 dataset = pd.get_dummies(dataset, prefix='', prefix_sep='') 
 dataset.tail()
 #Spliting the data into train and test
 #def split_dataset():
 train_dataset = dataset.sample(frac=0.8,random_state=0)
 test_dataset = dataset.drop(train_dataset.index)
 sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde") #Graph
 #overall statistics
 train_stats = train_dataset.describe()
 train_stats.pop("MPG")
 train_stats = train_stats.transpose()
 train_stats
 #Splitting features from labels
 train_labels = train_dataset.pop('MPG')
 test_labels = test_dataset.pop('MPG')
 return train_stats, train_labels, test_labels

#A
#Normalizing the data
def norm(x, train_stats): 
  return (x - train_stats['mean']) / train_stats['std']

#M
#Building the model
def build_model():
  model = keras.Sequential([layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]), layers.Dense(64, activation='relu'), layers.Dense(1)])
  optimizer = tf.keras.optimizers.RMSprop(0.001)
  model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
  return model

#A
#Inspecting the model
def inspect_model(model):
 model = build_model()
 model.summary()
 example_batch = normed_train_data[:10]
 example_result = model.predict(example_batch)
 return example_result

#M
#Training the model
def train_model(EP):
 history = model.fit(normed_train_data, train_labels, epochs=EP, validation_split = 0.2, verbose=0, callbacks=[tfdocs.modeling.EpochDots()])
 hist = pd.DataFrame(history.history)
 hist['epoch'] = history.epoch
 hist.tail()
 plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)
 plotter.plot({'Basic': history}, metric = "mae")
 plt.ylim([0, 10])
 plt.ylabel('MAE [MPG]')
 plotter.plot({'Basic': history}, metric = "mse")
 plt.ylim([0, 20])
 plt.ylabel('MSE [MPG^2]') 
 return plt.show()

#A
#Making prediction
def prediction():
 test_predictions = model.predict(normed_test_data).flatten()

 a = plt.axes(aspect='equal')
 plt.scatter(test_labels, test_predictions)
 plt.xlabel('True Values [MPG]')
 plt.ylabel('Predictions [MPG]')
 lims = [0, 50]
 plt.xlim(lims)
 plt.ylim(lims)
 _ = plt.plot(lims, lims)

 error = test_predictions - test_labels
 plt.hist(error, bins = 25)
 plt.xlabel("Prediction Error [MPG]")
 _ = plt.ylabel("Count")  
 return plt.show()
##-------------------------------------------------------------------
dataset_path = downloading_dataset(name_db, address_db)
train_stats1, train_dataset, test_dataset = preprocessing_dataset(dataset_path, column_names)
normed_train_data = norm(train_dataset, train_stats1) 
normed_test_data = norm(test_dataset, train_stats1)
#inspect_model(model_1)#---
model = build_model()
model.summary()
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

train_model(EPOCHS)#---

model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model.fit(normed_train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])
plotter.plot({'Early Stopping': early_history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [MPG]')

loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)
print("Testing set Loss: {:5.2f} MPG".format(loss))
#print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))
#print("Testing set mse: {:5.2f} MPG".format(mse))
prediction()

